In [24]:
import pandas as pd
data=pd.read_csv("FAOLEX.csv")
print(data.keys())

Index(['Record Id', 'Record URL', 'Document URL', 'Title', 'Original title',
       'Date of original text', 'Last amended date', 'Available website',
       'Language of document', 'Country/Territory', 'Regional organizations',
       'Territorial subdivision', 'Type of text', 'Repealed', 'Abstract',
       'Primary subjects', 'Domain', 'Keywords'],
      dtype='object')


In [35]:
import nltk
keywords=list(data["Keywords"])
country=list(data["Country/Territory"])
i=0
# Split alle keywords op i en liste og ændrer nan til at være en str med Nan
for words in keywords:
    if type(words)!=str:
        #print("type= ",type(words)," Index = ", i)
        words="Nan"
    if type(country[i])!=str:
        country[i]="NaN"
    keywords[i]=words.split(";")
    i+=1



In [26]:
#i=0

#for words in keywords:
   # keywords[i]=words + [" "+country[i]]
   # i+=1

In [36]:
# prints keywords for a single text
print(keywords[0])

['basic legislation', ' marine fisheries', ' navigation', ' foreign fishing', ' access right', ' fishing authorization', ' fishing gear/fishing method', ' offences/penalties', ' research', ' non-commercial fishing', ' concession', ' enforcement/compliance', ' fishing vessel', ' marking/identification', ' fishery management and conservation']


In [37]:
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import numpy as np
stemmer = SnowballStemmer("english")
stemmed_words=[]

# Stem keywords
for i in range(len(keywords)):
    stemmed_words.append([stemmer.stem(word) for word in keywords[i]])



In [38]:
print(keywords[14083])
print(stemmed_words[14083])

['basic legislation', ' land reform', ' land tenure', ' transfer\xa0rights\xa0', ' public land', ' land subdivision', ' right of use', ' expropriation', ' lease', ' common property', ' ownership', ' inheritance', ' cadastre/land registration', ' institution', ' agricultural land', ' land based credit', ' vested rights\xa0', ' farming', ' agricultural development', ' cooperative/producer organization', ' legal proceedings/administrative proceedings']
['basic legisl', ' land reform', ' land tenur', ' transfer\xa0rights\xa0', ' public land', ' land subdivis', ' right of us', ' expropri', ' leas', ' common properti', ' ownership', ' inherit', ' cadastre/land registr', ' institut', ' agricultural land', ' land based credit', ' vested rights\xa0', ' farm', ' agricultural develop', ' cooperative/producer organ', ' legal proceedings/administrative proceed']


In [14]:
# Find number of extinct domains
a= list(set(data["Domain"]))
i=0
for words in a:
    if type(words)!=str:
        #print("type= ",type(words)," Index = ", i)
        words="Nan"
    a[i]=words.split(";")

    i+=1



In [15]:
# print different domains
a=sum(a,[])
i=0
for word in a:
        word=word.strip()
        a[i]=word
        i+=1

print(set(a))




{'Energy', 'Nan', 'Fisheries', 'Land & soil', 'Wild species & ecosystems', 'Mineral resources', 'Livestock', 'Environment', 'Air & atmosphere', 'Sea', 'Cultivated plants', 'Water', 'Agriculture', 'Food & nutrition', 'General', 'Forestry', 'Waste & hazardous substances'}


In [39]:
# Count number documents in a specific language
languages=list(data["Language of document"])
print(languages.count("English"))

53564


In [42]:
# Make a top2vec topic model
from top2vec import Top2Vec
i=0
for word in keywords:
    keywords[i]="".join(word)
    i+=1
model = Top2Vec(keywords,embedding_model="doc2vec")


2023-05-29 11:52:16,779 - top2vec - INFO - Pre-processing documents for training
2023-05-29 11:52:23,569 - top2vec - INFO - Creating joint document/word embedding
2023-05-29 11:55:21,306 - top2vec - INFO - Creating lower dimension embedding of documents
2023-05-29 11:56:44,885 - top2vec - INFO - Finding dense areas of documents
2023-05-29 11:56:49,325 - top2vec - INFO - Finding topics


In [43]:
topic_sizes, topic_nums = model.get_topic_sizes()
topic_sizes

array([1733, 1566, 1345, ...,   20,   20,   18])

In [44]:
topic_words,word_scores,topic_nums = model.get_topics(10)

# Topic words: Ord i en given topic
#Words_score: Hvor 

In [46]:
print(topic_words)

[['seeds' 'floriculture' 'fibre' 'planting' 'plant' 'material' 'variety'
  'oleaginous' 'vegetables' 'breeders' 'legumes' 'fruits' 'tobacco'
  'nuts' 'cereals' 'edible' 'weeds' 'crops' 'rice' 'grains' 'viticulture'
  'beet' 'tea' 'oenological' 'cane' 'quarantine' 'cocoa' 'coffee'
  'certification' 'pests' 'sugar' 'intellectual' 'labelling' 'biological'
  'diseases' 'internal' 'commodities' 'packaging' 'production'
  'biotechnology' 'treatment' 'post' 'plants' 'harvest' 'extension'
  'oils' 'trade' 'procedures' 'oilseeds' 'genetic']
 ['mesh' 'fishing' 'fisheries' 'method' 'cartilaginous' 'charge'
  'fishery' 'catch' 'commercial' 'allowable' 'seasons' 'total' 'bycatch'
  'size' 'fishes' 'quota' 'crustaceans' 'gear' 'landing' 'diadromous'
  'artisanal' 'allocation' 'transshipment' 'foreign' 'vessel' 'molluscs'
  'prohibited' 'port' 'turtles' 'marine' 'mammals' 'migratory'
  'unreported' 'seas' 'repopulation' 'enhancement' 'high' 'stock' 'eez'
  'exclusive' 'state' 'illegal' 'unregulated' 

In [47]:
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=2, num_docs=5)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f"Document: {doc_id}, Score: {score}")
    print("-----------")
    print(doc)
    print("-----------")
    print()

Document: 712, Score: 0.9800017476081848
-----------
maritime zone territorial sea continental shelf EEZ-Exclusive Economic Zone offences/penalties sovereignty
-----------

Document: 4823, Score: 0.9796422123908997
-----------
maritime zone territorial sea EEZ-Exclusive Economic Zone continental shelf sovereignty
-----------

Document: 20938, Score: 0.9785271883010864
-----------
maritime zone territorial sea continental shelf EEZ-Exclusive Economic Zone navigation sovereignty
-----------

Document: 18706, Score: 0.9783092737197876
-----------
EEZ-Exclusive Economic Zone maritime zone territorial sea continental shelf sovereignty
-----------

Document: 23354, Score: 0.9763875603675842
-----------
maritime zone territorial sea continental shelf EEZ-Exclusive Economic Zone navigation sovereignty
-----------



In [50]:
topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=["maritime"], num_topics=3)
print(topic_words)
print(topic_nums)

[array(['maritime', 'territorial', 'sea', 'exclusive', 'continental',
       'eez', 'shelf', 'economic', 'zone', 'navigation', 'mesh', 'deep',
       'seas', 'sovereignty', 'fishing', 'diadromous', 'transshipment',
       'high', 'bycatch', 'harbour', 'catch', 'allowable', 'fisheries',
       'charge', 'total', 'foreign', 'marine', 'method', 'port',
       'fishery', 'vessel', 'cartilaginous', 'commercial', 'coastal',
       'landing', 'mammals', 'bed', 'artisanal', 'exploration', 'state',
       'unreported', 'quota', 'gear', 'fishes', 'migratory',
       'unregulated', 'illegal', 'mariculture', 'seasons', 'prohibited'],
      dtype='<U15'), array(['territorial', 'sea', 'maritime', 'exclusive', 'eez',
       'continental', 'shelf', 'seas', 'economic', 'high', 'mesh',
       'transshipment', 'fishing', 'zone', 'navigation', 'bycatch',
       'deep', 'port', 'harbour', 'charge', 'cartilaginous', 'catch',
       'foreign', 'method', 'marine', 'fisheries', 'vessel', 'diadromous',
       '

In [196]:
topic_mapping = model.hierarchical_topic_reduction(num_topics=20)

In [197]:
print(model.topic_words_reduced)

[['steroids' 'hormones' 'mrl' 'residue' 'drugs' 'limit' 'hygiene'
  'maximum' 'swine' 'procedures' 'residues' 'sanitary' 'feed'
  'feedstuffs' 'veterinary' 'animal' 'toxicity' 'poultry' 'poisoning'
  'equine' 'slaughtering' 'meat' 'cattle' 'sheep' 'vaccination' 'goats'
  'additives' 'health' 'dairy' 'mercosur' 'safety' 'disinfestation'
  'codex' 'inspection' 'rodents' 'treated' 'haccp' 'disinfection'
  'andean' 'edible' 'packaging' 'pests' 'traceability' 'trade'
  'antimicrobial' 'diseases' 'welfare' 'labelling' 'milk' 'alimentarius']
 ['equine' 'sheep' 'swine' 'goats' 'cattle' 'rodents' 'herd' 'camelids'
  'book' 'reproduction' 'poultry' 'animal' 'slaughtering' 'vaccination'
  'mercosur' 'animals' 'welfare' 'meat' 'health' 'disinfestation' 'birds'
  'jersey' 'hygiene' 'sericulture' 'apiculture' 'steroids' 'hormones'
  'drugs' 'procedures' 'disinfection' 'quarantine' 'identification'
  'north' 'pests' 'feed' 'sanitary' 'ranching' 'inspection' 'marking'
  'captive' 'diseases' 'edible' '

In [202]:
model.topic_words_reduced[4]

array(['viticulture', 'oenological', 'tribunal', 'court', 'herzegovina',
       'beverages', 'tobacco', 'geographical', 'bosnia', 'indication',
       'fraud', 'settlement', 'codex', 'inspection', 'lao',
       'compensation', 'dispute', 'intellectual', 'labelling', 'trade',
       'certification', 'justice', 'practices', 'consumer', 'lanka',
       'liability', 'packaging', 'deceit', 'information', 'plant',
       'haccp', 'traceability', 'alimentarius', 'offences', 'internal',
       'tristan', 'adulteration', 'classification', 'singapore',
       'breeders', 'access', 'seeds', 'commodities', 'vested', 'sugar',
       'floriculture', 'veterinary', 'liechtenstein', 'legal',
       'agricultural'], dtype='<U14')

In [82]:
topic_words,word_scores,topic_nums = model.get_topics(10)
topic_words

array([['mesh', 'fishing', 'seasons', 'size', 'fishery', 'method',
        'cartilaginous', 'catch', 'bycatch', 'fisheries', 'landing',
        'prohibited', 'marine', 'total', 'allowable', 'charge',
        'allocation', 'fishes', 'vessel', 'gear', 'quota', 'artisanal',
        'foreign', 'commercial', 'diadromous', 'port', 'migratory',
        'transshipment', 'turtles', 'mammals', 'molluscs', 'crustaceans',
        'illegal', 'high', 'seas', 'inland', 'iuu', 'territorial',
        'unregulated', 'area', 'exclusive', 'state', 'eez',
        'enhancement', 'management', 'conservation', 'coral',
        'unreported', 'repopulation', 'sea'],
       ['floriculture', 'planting', 'seeds', 'material', 'fibre',
        'plant', 'cane', 'beet', 'variety', 'vegetables', 'breeders',
        'oleaginous', 'legumes', 'fruits', 'edible', 'nuts', 'sugar',
        'certification', 'rice', 'tobacco', 'cereals', 'labelling',
        'weeds', 'crops', 'oenological', 'production', 'packaging',
        '

In [274]:
# Tæl hvor mange der er repelaed inden for et dato interval
dates=data["Date of original text"]
repealed=data["Repealed"]
antal=0
for i in range(1800,1970):
    c=dates==str(i)
    d=repealed=="Y"
    antal+=sum(c*d)
print(antal)


91


In [ ]:
model.generate_topic_wordcloud(2)
# 167 kunne godt være fødevare og børn